In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import numpy as np

In [2]:
# Read in all the data

AQMS_df=pd.read_csv('data/AQMS_readings.csv')
Rd_gdf=gpd.read_file('data/london_Road.shp')
Gsp_gdf=gpd.read_file('data/LD_GreenSpace.shp')
loc_gdf=gpd.read_file('data/AQMS_loc.shp')

In [3]:
AQMS_df

,Site,ReadingDateTime,Value
0,CT2,01/01/2019 00:00,13.0
1,CT2,01/01/2019 01:00,12.0
2,CT2,01/01/2019 02:00,8.0
3,CT2,01/01/2019 03:00,6.0
4,CT2,01/01/2019 04:00,5.0
...,...,...,...
210235,CD1,31/12/2019 19:00,22.4
210236,CD1,31/12/2019 20:00,23.8
210237,CD1,31/12/2019 21:00,21.8
210238,CD1,31/12/2019 22:00,23.1


In [ ]:
# reindex loc_gdf and set buffer zones around each site (1km)

loc_gdf=loc_gdf.set_index('siteid')
loc_gdf['buffer_1km']=loc_gdf['geometry'].buffer(1000)
loc_gdf

In [ ]:
# Get all green spaces

Gsp=Gsp_gdf['geometry'].unary_union
type(Gsp)

In [ ]:
# Set buffer zones for every road (50m)

Rd_gdf['buffer_50m']=Rd_gdf['geometry'].buffer(50)

In [ ]:
Rd=Rd_gdf['geometry'].unary_union

In [ ]:
# Get all road buffer zones

Rd_buffer=Rd_gdf['buffer_50m'].unary_union

In [ ]:
# Get all near-road green spaces

near_Rd_Gsp=Gsp.intersection(Rd_buffer)

In [ ]:
AQMS_shp=[]
for site in loc_gdf.index:
    AQMS_shp.append(loc_gdf.loc[site,'buffer_1km'])

In [ ]:
Rd_shp=[]
Gsp_shp=[]
near_Rd_Gsp_shp=[]
for poly in AQMS_shp:
    Rd_shp.append(poly.intersection(Rd))
    Gsp_shp.append(poly.intersection(Gsp))
    near_Rd_Gsp_shp.append(poly.intersection(near_Rd_Gsp))
print(len(Rd_shp),len(Gsp_shp),len(near_Rd_Gsp_shp))

In [ ]:
loc_gdf['Road']=Rd_shp
loc_gdf['GreenSpace']=Gsp_shp
loc_gdf['NR_GreenSpace']=near_Rd_Gsp_shp
loc_gdf=loc_gdf.set_geometry('Road').set_geometry('GreenSpace').set_geometry('NR_GreenSpace')
loc_gdf

In [ ]:
loc_gdf.dtypes

In [ ]:
london=gpd.read_file('data/london_boundary.shp')

In [ ]:
fig,ax=plt.subplots(1,figsize=(15,13))

london.plot(color='lightgrey',ax=ax)
loc_gdf['buffer_1km'].plot(color='silver',ax=ax)
loc_gdf['GreenSpace'].plot(label='Green space',color='limegreen',ax=ax)
loc_gdf['NR_GreenSpace'].plot(label='Near_road green space',color='darkgreen',ax=ax)
loc_gdf['Road'].plot(color='black',label='Road',linewidth=0.1,ax=ax)
loc_gdf['geometry'].plot(markersize=5,marker='^',color='blue',label='Air quality monitoring site',ax=ax)

ax.axis('off')

legend=ax.legend(loc='best',shadow=True,fontsize=15)

plt.savefig('sample1.png',facecolor='black',dpi=500)
plt.show()